# download and load (Steinmetz et al 2019)

original loading notebook from: https://compneuro.neuromatch.io/projects/neurons/README.html#steinmetz

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore, ranksums
from sklearn.decomposition import PCA
from rastermap import Rastermap, utils
from tqdm import trange 

sys.path.insert(0, "/github/rastermap/paper")
import fig6

# path to directory with data etc
### *** CHANGE THIS TO WHEREEVER YOU ARE SAVING YOUR DATA OUTPUTS ***
root = "/media/carsen/ssd2/rastermap_paper/"

# Data retrieval
import os, requests

fname = []
for j in range(3):
  fname.append(os.path.join(root, "data", "steinmetz_part%d.npz"%j))
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

for j in range(len(url)):
  if not os.path.isfile(fname[j]):
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fname[j], "wb") as fid:
          fid.write(r.content)

# Data loading
alldat = np.array([])
for j in range(len(fname)):
  alldat = np.hstack((alldat,
                      np.load(os.path.join(root, "data", "steinmetz_part%d.npz"%j),
                              allow_pickle=True)['dat']))

### sort trials from each recording

In [ ]:
psigs = []
brain_areas = []
reaction_times = []
pupil_speeds = []
licks = []
isorts = []
itrials = []
rewards = []
face_motions = []
wheel_moves = []
ccfs = []
# 6
for d in trange(len(alldat)):
    spks = alldat[d]["spks"].copy().astype("float32")
    nn,_,nt = spks.shape
    brain_area = alldat[d]["brain_area"]
    response = alldat[d]["response"].flatten()
    rtypes = [-1, 1]
    ttypes = ["stim", "reaction", "feedback"]

    spks = spks.reshape(nn, -1)
    igood = ((spks.mean(axis=-1)) / .01) > 0.1
    igood *= (brain_area != "root")
    spks = zscore(spks[igood], axis=-1)
    brain_area = brain_area[igood]
    ccfs.append(alldat[d]["ccf"][igood])

    n_PCs = 10
    spcs = PCA(n_components=n_PCs).fit_transform(spks.T).T
    U = spks @ (spcs.T / (spcs**2).sum(axis=1)**0.5)
    spcs_trials = spcs.reshape(n_PCs, -1, nt)
    spks = spks.reshape(spks.shape[0], -1, nt)

    brain_areas.append(brain_area)
    for r in [0, 1]:
        rtype = rtypes[r]
        if rtype != 0:
            itrial = np.logical_and(response==rtype, ~np.isinf(alldat[d]["reaction_time"][:,0]))
        else:
            itrial = response==rtype
        ntrials = itrial.sum()
        
        sresp = spcs_trials[:,itrial].copy()
        sresp -= sresp.mean(axis=(-2,-1), keepdims=True)
        sresp_std = sresp.std(axis=(-2,-1), keepdims=True)
        sresp /= sresp_std

        # flatten across PCs to sort trials
        sresp_trials = sresp.copy().transpose(1,0,2).reshape(ntrials, -1)
        
        model = Rastermap(n_clusters=None, n_PCs=64, 
                        time_lag_window=0, time_bin=0,
                        locality=0.1, mean_time=False, verbose=False
                        ).fit(sresp_trials)
        
        isorts.append(model.isort)
        itrials.append(itrial)
        pspeed = (np.diff(alldat[d]["pupil"][1:], axis=-1)**2).sum(axis=0)**0.5
        pupil_speeds.append(pspeed[itrial][model.isort].mean(axis=-1))
        wms = alldat[d]["wheel"].squeeze()[itrial][model.isort].mean(axis=-1)
        wheel_moves.append(wms)
        rts = alldat[d]["reaction_time"][itrial,0][model.isort]/1000
        reaction_times.append(rts)
        lcks = alldat[d]["licks"][0,itrial][model.isort].sum(axis=-1)
        licks.append(lcks)
        rwds = alldat[d]["feedback_type"][itrial][model.isort]
        rewards.append(rwds)
        fe = alldat[d]["face"][0,itrial][model.isort].mean(axis=-1)
        face_motions.append(fe)
        iswap = model.isort[:10].mean() > model.isort[-10:].mean()

        #ss = spks[:,itrial].copy()
        ss = alldat[d]["spks"][igood][:,itrial].copy()
        ss = ss.mean(axis=-1)
        s0 = ss[:,:20]
        s1 = ss[:,-20:]

        stats, p = ranksums(s0.T, s1.T)
        psig = np.sign(stats) * (p<0.05)
        psigs.append(psig)

### compute percentage of (p<0.05) of late vs early per brain area

In [ ]:
brain_groups = [["VISa", "VISam", "VISl", "VISp", "VISpm", "VISrl"], # visual cortex
                ["CL", "LD", "LGd", "LH", "LP", "MD", "MG", "PO", "POL", "PT", "RT", "SPF", "TH", "VAL", "VPL", "VPM"], # thalamus
                ["CA", "CA1", "CA2", "CA3", "DG", "SUB", "POST"], # hippocampal
                ["ACA", "AUD", "COA", "DP", "ILA", "MOp", "MOs", "OLF", "ORB", "ORBm", "PIR", "PL", "SSp", "SSs", "RSP","TT"], # non-visual cortex
                ["APN", "IC", "MB", "MRN", "NB", "PAG", "RN", "SCs", "SCm", "SCig", "SCsg", "SNr", "ZI"], # midbrain
                ["ACB", "CP", "GPe", "LS", "LSc", "LSr", "MS", "OT", "SI"], # basal ganglia 
                ["BLA", "BMA", "EP", "EPd", "MEA"] # cortical subplate
               ]

area = ["vis ctx", "thalamus", "hippocampal", "other ctx", "midbrain", "striatum", "amygdala"]

perc = np.zeros((len(psigs), 3, 7))
import itertools
ccf_all = list(itertools.chain(*ccfs))
for k in range(len(psigs)):
    bass = np.zeros(len(brain_areas[k//2]), "int")
    for i,b in enumerate(brain_groups):
        bind = np.isin(brain_areas[k//2], b)
        bass[bind] = i
    perc[k, 0] = np.histogram(bass[psigs[k] > 0], np.arange(0, 8))[0]
    perc[k, 1] = np.histogram(bass[psigs[k] < 0], np.arange(0, 8))[0]
    perc[k, 2] = np.histogram(bass[np.abs(psigs[k]) > 0], np.arange(0, 8))[0]
    nb = np.histogram(bass, np.arange(0, 8))[0]
    perc[k] /= nb 
perc *= 100


In [ ]:
# save results
np.savez(os.path.join(root, "results/steinmetz_proc.npz"), 
         ccf_all=ccf_all, itrials=np.array(itrials, dtype=object), isorts=np.array(isorts, dtype=object), 
         reaction_times=np.array(reaction_times, dtype=object), rewards=np.array(rewards, dtype=object), 
         licks=np.array(licks, dtype=object), wheel_moves=np.array(wheel_moves, dtype=object), 
         face_motions=np.array(face_motions, dtype=object), pupil_speeds=np.array(pupil_speeds, dtype=object), 
         perc=perc, area=np.array(area))

In [ ]:
# will save figures to "figures" folder
os.makedirs(os.path.join(root, "figures/"), exist_ok=True)
fig6.fig6(root, alldat)